# Visualizing High-dimensional Parameter Relationships

This notebook demonstrates various visualizations of studies in Optuna.
The hyperparameters of a neural networks trained to classify images are optimized and the resulting study is then visualized using these features.

**Note:** If a parameter contains missing values, a trial with missing values is not plotted.

## Preparing the Dataset

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml(name='Fashion-MNIST', version=1)
classes = list(set(mnist.target))
x_train, x_test, y_train, y_test = train_test_split(mnist.data, mnist.target)

## Defining the Objective Function

In [ ]:
from sklearn.neural_network import MLPClassifier

def objective(trial):
    layers = []
    n_layers = trial.suggest_int('n_layers', 1, 3)
    for i in range(n_layers):
        layers.append(trial.suggest_int('n_units_l{}'.format(i), 32, 64))

    clf = MLPClassifier(hidden_layer_sizes=tuple(layers))

    for step in range(100):
        clf.partial_fit(x_train, y_train, classes=classes)

        intermediate_value = 1.0 - clf.score(x_test, y_test)  # Report intermediate objective value.
        trial.report(intermediate_value, step)

        if trial.should_prune(step):
            raise optuna.exceptions.TrialPruned()  # Handle pruning based on the intermediate value.

    return clf.score(x_test, y_test)

## Running the Optimization

In [ ]:
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)  # This verbosity change is just to simplify the notebook output.

study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=32)

## Visualizing the Optimization History

In [ ]:
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

## Visualizing the Learning Curves of the Trials

In [ ]:
from optuna.visualization import plot_intermediate_values

plot_intermediate_values(study)

## Visualizing High-dimensional Parameter Relationships

In [ ]:
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study)

### Selecting parameters to Visualize

In [ ]:
plot_parallel_coordinate(study, params=["n_layers", "n_units_l1"])

## Visualizing Hyper-parameter Relationships

In [ ]:
from optuna.visualization import plot_contour

plot_contour(study)

### Selecting parameters to Visualize

In [ ]:
plot_contour(study, params=["n_units_l0", "n_units_l1"])

## Visualizing Individual Hyper-parameters

In [ ]:
from optuna.visualization import plot_slice

plot_slice(study)

### Selecting parameters to Visualize

In [ ]:
plot_slice(study, params=["n_units_l0", "n_units_l1"])